In [1]:
import pandas as pd 
import numpy as np 

In [3]:
country = 'mwi_mics19-20'

In [ ]:
data = pd.read_csv('./'+ country)

In [11]:
data

,hh_id,psu,strata,weight,d_cm,d_nutr,d_satt,d_educ,d_elct,d_sani,...,cookingfuel,television,radio,telephone,refrigerator,car,bicycle,motorbike,animal_cart,computer
0,101,1,Chitipa Rural,0.432607,0.0,NaN,0.0,0.0,1.0,0.0,...,WOOD,No,No,No,No,No,No,No,No,No
1,102,1,Chitipa Rural,0.432607,0.0,1.0,0.0,1.0,1.0,1.0,...,WOOD,No,No,Yes,No,No,Yes,No,No,No
2,103,1,Chitipa Rural,0.432607,0.0,1.0,0.0,0.0,1.0,1.0,...,WOOD,No,No,Yes,No,No,No,No,No,No
3,104,1,Chitipa Rural,0.432607,0.0,0.0,0.0,0.0,1.0,0.0,...,WOOD,No,Yes,Yes,No,No,Yes,No,No,No
4,106,1,Chitipa Rural,0.432607,0.0,0.0,0.0,1.0,1.0,1.0,...,WOOD,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25414,111220,1112,Blantyre City,1.204571,0.0,0.0,0.0,1.0,1.0,1.0,...,WOOD,No,Yes,No,No,No,No,No,No,No
25415,111221,1112,Blantyre City,1.204571,0.0,1.0,0.0,0.0,1.0,1.0,...,WOOD,Yes,Yes,Yes,No,No,Yes,No,No,No
25416,111222,1112,Blantyre City,1.204571,0.0,0.0,1.0,0.0,1.0,1.0,...,CHARCOAL,Yes,Yes,Yes,Yes,No,No,Yes,No,No
25417,111223,1112,Blantyre City,1.204571,0.0,0.0,0.0,0.0,1.0,1.0,...,WOOD,No,No,No,No,No,No,No,No,No


In [13]:
PSU = data.groupby('psu').agg({'weight' : 'max' ,'strata' : 'max', 'region' : 'max'}).reset_index()

In [14]:
# Convert variables to correct data type:
for col in data.columns:
    data[col] = data[col].astype('category')

In [15]:
data['weight'] = data.weight.astype('float')

In [18]:
from mostlyai import engine
from pathlib import Path

In [19]:
ws = Path("temp-ws")
engine.init_logging()


In [20]:
# Split data
engine.split(                         # split data as PQT files for trn + val to {ws}/OriginalData/tgt-data
    workspace_dir=ws,   
    tgt_data=data,
    ctx_data=PSU,
    tgt_primary_key='hh_id',
    ctx_primary_key='psu',
    tgt_context_key='psu',
    model_type="TABULAR"
    )


[2025-07-18 01:18:23,097] INFO   : SPLIT started
[2025-07-18 01:18:23,103] INFO   : clean `temp-ws/OriginalData/tgt-data`
[2025-07-18 01:18:23,105] INFO   : clean `temp-ws/OriginalData/tgt-meta`
[2025-07-18 01:18:23,106] INFO   : create `temp-ws/OriginalData/ctx-data`
[2025-07-18 01:18:23,107] INFO   : create `temp-ws/OriginalData/ctx-meta`
[2025-07-18 01:18:23,115] INFO   : model_type='TABULAR'
[2025-07-18 01:18:23,116] INFO   : tgt_encoding_types={'strata': 'TABULAR_CATEGORICAL', 'weight': 'TABULAR_NUMERIC_AUTO', 'd_cm': 'TABULAR_CATEGORICAL', 'd_nutr': 'TABULAR_CATEGORICAL', 'd_satt': 'TABULAR_CATEGORICAL', 'd_educ': 'TABULAR_CATEGORICAL', 'd_elct': 'TABULAR_CATEGORICAL', 'd_sani': 'TABULAR_CATEGORICAL', 'd_hsg': 'TABULAR_CATEGORICAL', 'd_ckfl': 'TABULAR_CATEGORICAL', 'd_asst': 'TABULAR_CATEGORICAL', 'sex_Male': 'TABULAR_CATEGORICAL', 'sex_Female': 'TABULAR_CATEGORICAL', 'agec7_0-4': 'TABULAR_CATEGORICAL', 'agec7_5-9': 'TABULAR_CATEGORICAL', 'agec7_10-14': 'TABULAR_CATEGORICAL', 'ag

In [21]:
engine.analyze(workspace_dir=ws,
            value_protection = False)    # Avoid value protetion


[2025-07-18 01:18:39,512] INFO   : ANALYZE started
[2025-07-18 01:18:39,516] INFO   : clean `temp-ws/ModelStore/tgt-stats`
[2025-07-18 01:18:39,517] INFO   : create `temp-ws/ModelStore/ctx-stats`
[2025-07-18 01:18:39,519] INFO   : analyzing 2 partitions in parallel
[2025-07-18 01:18:43,208] INFO   : analyzed target partition 000000-trn (20370, 54)
[2025-07-18 01:18:43,274] INFO   : analyzed context partition 000000-trn (889, 4)
[2025-07-18 01:18:43,523] INFO   : analyzed target partition 000000-val (5049, 54)
[2025-07-18 01:18:43,574] INFO   : analyzed context partition 000000-val (222, 4)
[2025-07-18 01:18:43,575] INFO   : combine partition statistics
[2025-07-18 01:18:43,581] INFO   : value_protection = False
[2025-07-18 01:18:43,582] INFO   : analyzed column `strata`: TABULAR_CATEGORICAL {'cat': 57}
[2025-07-18 01:18:43,584] INFO   : analyzed column `weight`: TABULAR_NUMERIC_BINNED {'bin': 103}
[2025-07-18 01:18:43,585] INFO   : analyzed column `d_cm`: TABULAR_CATEGORICAL {'cat': 4}

In [22]:
engine.encode(workspace_dir=ws) 

[2025-07-18 01:18:54,194] INFO   : ENCODE_TABULAR started
[2025-07-18 01:18:54,196] INFO   : clean `temp-ws/OriginalData/encoded-data`
[2025-07-18 01:18:54,197] INFO   : clean `temp-ws/OriginalData/encoded-data`
[2025-07-18 01:18:54,198] INFO   : clean `temp-ws/OriginalData/encoded-data`
[2025-07-18 01:18:55,065] INFO   : encoded partition part.000000-trn.parquet (889, 58)
[2025-07-18 01:18:55,381] INFO   : encoded partition part.000000-val.parquet (222, 58)
[2025-07-18 01:18:55,385] INFO   : ENCODE_TABULAR finished in 1.19s


In [ ]:
    
engine.train(                         # train model and store to {ws}/ModelStore/model-data
    workspace_dir=ws,
    model = "MOSTLY_AI/Large",
    max_training_time=10,              # limit TRAIN to 1 minute for demo purposes
    device = 'cuda'  # Use GPU if available, 
    )

[2025-07-18 01:19:07,385] INFO   : TRAIN_TABULAR started
[2025-07-18 01:19:07,404] INFO   : numpy=2.3.1, pandas=2.2.3
[2025-07-18 01:19:07,406] INFO   : torch=2.7.0, opacus=1.5.4
[2025-07-18 01:19:07,406] INFO   : device=device(type='cuda')
[2025-07-18 01:19:07,408] INFO   : is_sequential=True
[2025-07-18 01:19:07,408] INFO   : max_training_time=600s
[2025-07-18 01:19:07,409] INFO   : max_epochs=100.0 -> max_epochs=23 due to small sample size
[2025-07-18 01:19:07,409] INFO   : model_size=<ModelSize.L: 'L'>
[2025-07-18 01:19:07,410] INFO   : enable_flexible_generation=True
[2025-07-18 01:19:07,410] INFO   : with_dp=False
[2025-07-18 01:19:07,411] INFO   : model_state_strategy=<ModelStateStrategy.reset: 'RESET'>
[2025-07-18 01:19:07,413] INFO   : max_sequence_window=np.int64(56)
[2025-07-18 01:19:07,413] INFO   : create training model
[2025-07-18 01:19:07,917] INFO   : model class: SequentialModel
[2025-07-18 01:19:07,918] INFO   : model_state_strategy=<ModelStateStrategy.reset: 'RESET'>

In [ ]:
engine.generate(workspace_dir=ws) 

In [ ]:
gen_df = pd.read_parquet(ws / "SyntheticData")

In [ ]:
gen_df.to_csv('./syn_'+country)